***
***
# __Update 3__
### *Adam Becklehimer*
***
***

In [1]:
from __future__ import division
import concurrent.futures
import pandas as pd
import numpy as np
import re
from scipy import stats


from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import precision_recall_fscore_support
from joblib import Parallel, delayed
import multiprocessing
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
#from sklearn.preprocessing import LabelEncoder
#from sklearn.preprocessing import OneHotEncoder
#from sklearn import preprocessing
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, Flatten, MaxPooling1D

In [3]:
from collections import Counter
from imblearn.over_sampling import SMOTE

***
***
## Setup
***
***


In [4]:
def runs(lst):
	#"Iterator, chunks repeated values"
    for j,two in enumerate(lst):
        if j == 0:
            one,i = two,0
        if one!=two:
            yield j - i,one
            i = j
        one=two
    yield j - i + 1,two


def cliffsDelta(lst1,lst2,dull = [0.147, 0.33, 0.474, ][0] ):
    #"Returns true if there are more than 'dull' differences"
    m, n = len(lst1), len(lst2)
    lst2 = sorted(lst2)
    j = more = less = 0
    for repeats,x in runs(sorted(lst1)):
        while j <= (n - 1) and lst2[j] <  x:
            j += 1
        more += j*repeats
        while j <= (n - 1) and lst2[j] == x:
            j += 1
        less += (n - j)*repeats
        d= (more - less) / (m*n)
	# return abs(d)  > dull
    return round(abs(d), 3) ### return the clacluated cliffs delat
	

def _cliffsDelta():
	#"demo function"
    lst1=[1,2,3,4,5,6,7]
    for r in [1.01,1.1,1.21, 1.5, 2]:
        lst2=map(lambda x: x*r,lst1)
        print (r,cliffsDelta(lst1,lst2)) # should return False
df_mann= list()
def compareLists(defective_vals_for_feature, non_defective_vals_for_feature):

        print ('='*25)
        print ("Defective values [MEDIAN]:{}, [MEAN]:{}, [COUNT]:{}".format(np.median(list(defective_vals_for_feature)), np.mean(list(defective_vals_for_feature)), len(defective_vals_for_feature)))
        print ("Non Defective values [MEDIAN]:{}, [MEAN]:{}, [COUNT]:{}".format(np.median(list(non_defective_vals_for_feature)), np.mean(list(non_defective_vals_for_feature)), len(non_defective_vals_for_feature)))
        try:
            TS, p = stats.mannwhitneyu(list(defective_vals_for_feature), list(non_defective_vals_for_feature), alternative='greater')
        except ValueError:
            TS, p = 0.0, 1.0
        cliffs_delta = cliffsDelta(list(defective_vals_for_feature), list(non_defective_vals_for_feature))
        df_mann.append((p,cliffs_delta))
        print ('*'*25)
        #print ('Feature:{}, pee value:{}, cliffs:{}'.format(feature_, p, cliffs_delta))
        print ('p value:{}, cliffs:{}'.format(p, cliffs_delta))
        print ('*'*25)
        print ('='*50)

In [5]:
executor = concurrent.futures.ThreadPoolExecutor(max_workers=28)

def read():
    df_ = pd.read_csv('repo_secu.csv', engine = 'python')
    return df_

a = executor.submit(read)

print(a)

<Future at 0x7ffb1c5095f8 state=running>


In [6]:
print(a)

<Future at 0x7ffb1c5095f8 state=running>


In [7]:
df_ = a.result()
#print(df_)
df_.REPO_TYPE.unique()

array(['ComputationalChemistry', 'Astronomy', 'ComputationalBiology',
       'ComputationalScience', 'ComputationalIntelligence'], dtype=object)

In [8]:
df_comp_bio = (df_[df_['REPO_TYPE']== 'ComputationalBiology'])
df_astronomy = (df_[df_['REPO_TYPE'] == 'Astronomy'])
df_comp_chem = (df_[df_['REPO_TYPE'] == 'ComputationalChemistry'])
df_comp_sci = (df_[df_['REPO_TYPE'] == 'ComputationalScience'])
df_comp_intel = (df_[df_['REPO_TYPE'] == 'ComputationalIntelligence'])
df_others = pd.concat([df_comp_sci, df_comp_intel], axis=0)


df_comp_bio_full = df_comp_bio[["PRIOR_AGE","ADD_LOC","DEL_LOC","TOT_LOC", "DEV_EXP", "DEV_RECENT" , "CHANGE_FILE_CNT", "SECU_FLAG"]]
df_astro_full = df_astronomy[["PRIOR_AGE","ADD_LOC","DEL_LOC","TOT_LOC", "DEV_EXP", "DEV_RECENT",  "CHANGE_FILE_CNT", "SECU_FLAG"]]
df_comp_chem_full = df_comp_chem[["PRIOR_AGE","ADD_LOC","DEL_LOC","TOT_LOC", "DEV_EXP", "DEV_RECENT" , "CHANGE_FILE_CNT", "SECU_FLAG"]]
df_others_full = df_others[["PRIOR_AGE","ADD_LOC","DEL_LOC","TOT_LOC", "DEV_EXP", "DEV_RECENT",  "CHANGE_FILE_CNT", "SECU_FLAG"]]

***
***
## Mann - Whitney U Test and Effect Size
***
***

In [9]:
def Mann (model,feature):
    valid_features =[]
    invalid_features =[]
    for index, row in model.iterrows():
        if(row['SECU_FLAG'] == "NEUTRAL"):
            valid_features.append(row[feature])
        else:
            invalid_features.append(row[feature])
   
    #print(str(model))
    print("  ")
    print( feature)
    print("\n")
    compareLists(invalid_features,valid_features)
    return 0

In [10]:
print("MAN WHITNEY U TEST FOR df_others")
Mann(df_others_full,"ADD_LOC")
print("MAN WHITNEY U TEST FOR df_comp_bio_full")
Mann(df_comp_bio_full,"ADD_LOC")
print("MAN WHITNEY U TEST FOR df_astro_full")
Mann(df_astro_full,"ADD_LOC")
print("MAN WHITNEY U TEST FOR df_comp_chem_full")
Mann(df_comp_chem_full,"ADD_LOC")
print("\n\n\n\n")

print("MAN WHITNEY U TEST FOR df_others")
Mann(df_others_full,"PRIOR_AGE")
print("MAN WHITNEY U TEST FOR df_comp_bio_full")
Mann(df_comp_bio_full,"PRIOR_AGE")
print("MAN WHITNEY U TEST FOR df_astro_full")
Mann(df_astro_full,"PRIOR_AGE")
print("MAN WHITNEY U TEST FOR df_comp_chem_full")
Mann(df_comp_chem_full,"PRIOR_AGE")
print("\n\n\n\n")

print("MAN WHITNEY U TEST FOR df_others")
Mann(df_others_full,"DEL_LOC")
print("MAN WHITNEY U TEST FOR df_comp_bio_full")
Mann(df_comp_bio_full,"DEL_LOC")
print("MAN WHITNEY U TEST FOR df_astro_full")
Mann(df_astro_full,"DEL_LOC")
print("MAN WHITNEY U TEST FOR df_comp_chem_full")
Mann(df_comp_chem_full,"DEL_LOC")
print("\n\n\n\n")


print("MAN WHITNEY U TEST FOR df_others")
Mann(df_others_full,"TOT_LOC")
print("MAN WHITNEY U TEST FOR df_comp_bio_full")
Mann(df_comp_bio_full,"TOT_LOC")
print("MAN WHITNEY U TEST FOR df_astro_full")
Mann(df_astro_full,"TOT_LOC")
print("MAN WHITNEY U TEST FOR df_comp_chem_full")
Mann(df_comp_chem_full,"TOT_LOC")
print("\n\n\n\n")

print("MAN WHITNEY U TEST FOR df_others")
Mann(df_others_full,"DEV_EXP")
print("MAN WHITNEY U TEST FOR df_comp_bio_full")
Mann(df_comp_bio_full,"DEV_EXP")
print("MAN WHITNEY U TEST FOR df_astro_full")
Mann(df_astro_full,"DEV_EXP")
print("MAN WHITNEY U TEST FOR df_comp_chem_full")
Mann(df_comp_chem_full,"DEV_EXP")
print("\n\n\n\n")

print("MAN WHITNEY U TEST FOR df_others")
Mann(df_others_full,"DEV_RECENT")
print("MAN WHITNEY U TEST FOR df_comp_bio_full")
Mann(df_comp_bio_full,"DEV_RECENT")
print("MAN WHITNEY U TEST FOR df_astro_full")
Mann(df_astro_full,"DEV_RECENT")
print("MAN WHITNEY U TEST FOR df_comp_chem_full")
Mann(df_comp_chem_full,"DEV_RECENT")
print("\n\n\n\n")



print("MAN WHITNEY U TEST FOR df_others")
Mann(df_others_full,"CHANGE_FILE_CNT")
print("MAN WHITNEY U TEST FOR df_comp_bio_full")
Mann(df_comp_bio_full,"CHANGE_FILE_CNT")
print("MAN WHITNEY U TEST FOR df_astro_full")
Mann(df_astro_full,"CHANGE_FILE_CNT")
print("MAN WHITNEY U TEST FOR df_comp_chem_full")
Mann(df_comp_chem_full,"CHANGE_FILE_CNT")
print("\n\n\n\n")

MAN WHITNEY U TEST FOR df_others
  
ADD_LOC


Defective values [MEDIAN]:49.0, [MEAN]:360.4206008583691, [COUNT]:233
Non Defective values [MEDIAN]:23.0, [MEAN]:593.3756552125801, [COUNT]:5151
*************************
p value:5.4656250954923225e-06, cliffs:0.17
*************************
MAN WHITNEY U TEST FOR df_comp_bio_full
  
ADD_LOC


Defective values [MEDIAN]:16.0, [MEAN]:524.2652541833684, [COUNT]:15717
Non Defective values [MEDIAN]:11.0, [MEAN]:663.8411968646367, [COUNT]:159216
*************************
p value:7.19411106581891e-82, cliffs:0.092
*************************
MAN WHITNEY U TEST FOR df_astro_full
  
ADD_LOC


Defective values [MEDIAN]:14.0, [MEAN]:146.26299504950495, [COUNT]:4848
Non Defective values [MEDIAN]:11.0, [MEAN]:307.9216749871992, [COUNT]:62496
*************************
p value:2.4756092002005906e-16, cliffs:0.07
*************************
MAN WHITNEY U TEST FOR df_comp_chem_full
  
ADD_LOC


Defective values [MEDIAN]:28.0, [MEAN]:1583.1567711807872, [COUNT]:2

  
DEV_RECENT


Defective values [MEDIAN]:253.1369, [MEAN]:378.27465767120464, [COUNT]:4848
Non Defective values [MEDIAN]:272.16667, [MEAN]:419.5081930446749, [COUNT]:62496
*************************
p value:0.9999999999999996, cliffs:0.069
*************************
MAN WHITNEY U TEST FOR df_comp_chem_full
  
DEV_RECENT


Defective values [MEDIAN]:226.35476, [MEAN]:229.05166287191457, [COUNT]:2998
Non Defective values [MEDIAN]:200.50238, [MEAN]:256.8828277733466, [COUNT]:49397
*************************
p value:0.9964186493408792, cliffs:0.029
*************************





MAN WHITNEY U TEST FOR df_others
  
CHANGE_FILE_CNT


Defective values [MEDIAN]:2.0, [MEAN]:24.510729613733904, [COUNT]:233
Non Defective values [MEDIAN]:2.0, [MEAN]:12.26227916909338, [COUNT]:5151
*************************
p value:0.02750644740917249, cliffs:0.072
*************************
MAN WHITNEY U TEST FOR df_comp_bio_full
  
CHANGE_FILE_CNT


Defective values [MEDIAN]:2.0, [MEAN]:7.046764649742317, [COUNT]:157

In [11]:
row = list()
cols  = ["ADD_LOC","PRIOR_AGE","DEL_LOC","TOT_LOC", "DEV_EXP", "DEV_RECENT" , "CHANGE_FILE_CNT", "SECU_FLAG"]
j = 0
k =0
for i in range(7):
    row.append('OTHERS_' +cols[j])
    row.append('BIO_' + cols[j])
    row.append('ASTRO_' + cols[j])
    row.append('CHEM_' + cols[j])
    j+=1

df = pd.DataFrame(df_mann,columns = ['p-value','cliff_delta'],index=row)

    

In [12]:
df

,p-value,cliff_delta
OTHERSADD_LOC,5.465625e-06,0.170
BIOADD_LOC,7.194111e-82,0.092
ASTROADD_LOC,2.475609e-16,0.070
CHEMADD_LOC,2.033629e-26,0.115
OTHERSPRIOR_AGE,9.981343e-01,0.112
BIOPRIOR_AGE,9.807043e-01,0.010
ASTROPRIOR_AGE,2.788020e-01,0.005
CHEMPRIOR_AGE,9.673686e-01,0.020
OTHERSDEL_LOC,1.473033e-03,0.115
BIODEL_LOC,5.876286e-11,0.031


In [13]:
df.to_csv('mwu.csv')

***
***
# General Transfer Learning
***
***

In [14]:
def produce_metrics(y_actual, y_predicted):
    
    rounded = list(np.around(np.array(precision_recall_fscore_support(y_actual, y_predicted,average = None)),2))
    print(rounded)
    return (rounded)

In [15]:
def run_transfer_model(model, train_X, train_y, target_X, target_y):
    
    train_y = np.ravel(train_y)
    target_y = np.ravel(target_y)
    
    model.fit(train_X, train_y)
    y_predi = model.predict(target_X)
    metrics = produce_metrics(target_y, y_predi)
    print(confusion_matrix(target_y, y_predi))
    return (metrics)

In [16]:
Bio_Xy = df_comp_bio_full.to_numpy()
Bio_X = Bio_Xy[:,0:7]
Bio_y = Bio_Xy[:,7]

Astro_Xy = df_astro_full.to_numpy()
Astro_X = Astro_Xy[:,0:7]
Astro_y = Astro_Xy[:,7]

Chem_Xy = df_comp_chem_full.to_numpy()
Chem_X = Chem_Xy[:,0:7]
Chem_y = Chem_Xy[:,7]

Others_Xy = df_others_full.to_numpy()
Others_X = Others_Xy[:,0:7]
Others_y = Others_Xy[:,7]


#le = preprocessing.LabelEncoder()
#
#Bio_y = le.fit_transform(Bio_y)
#Astro_y = le.fit_transform(Astro_y)

Bio_y = label_binarize(np.ravel(Bio_y), classes=['NEUTRAL', 'INSECURE'])
Astro_y = label_binarize(np.ravel(Astro_y), classes=['NEUTRAL', 'INSECURE'])
Chem_y = label_binarize(np.ravel(Chem_y), classes=['NEUTRAL', 'INSECURE'])
Others_y = label_binarize(np.ravel(Others_y), classes=['NEUTRAL', 'INSECURE'])

print(Bio_y.shape, Astro_y.shape, Chem_y.shape, Others_y.shape)


(174933, 1) (67344, 1) (52395, 1) (5384, 1)


***
***
# SMOTE Application
***
***

In [17]:
sm = SMOTE(random_state=42)
Bio_X_res, Bio_y_res = sm.fit_resample(Bio_X, np.ravel(Bio_y))
Astro_X_res, Astro_y_res = sm.fit_resample(Astro_X, np.ravel(Astro_y))
Chem_X_res, Chem_y_res = sm.fit_resample(Chem_X, np.ravel(Chem_y))
Others_X_res, Others_y_res = sm.fit_resample(Others_X, np.ravel(Others_y))

In [18]:
df_metrics = pd.DataFrame( columns=['Train_Set', 'Target_Set', 'Model_Type','Neutral_Precision','Insecure_Precision', 'Neutral_Recall', 'Insecure_Recall', 
                                              'Neutral_F', 'Insecure_F', 'Neutral_Support', 'Insecure_Support', 'Neutral_Precision_SMOTE','Insecure_Precision_SMOTE',
                                              'Neutral_Recall_SMOTE', 'Insecure_Recall_SMOTE', 'Neutral_F_SMOTE', 'Insecure_F_SMOTE', 'Neutral_Support_SMOTE', 'Insecure_Support_SMOTE' ])

df_metrics
#put new entry into table
def add_metrics_row(trainer, tester, clf_type, normals, smotes, df_):
    new_row = [trainer, tester, clf_type]
    for x in normals:
        for y in x:
            new_row.append(y)    
    for x in smotes:
        for y in x:
            new_row.append(y)        
    df_.loc[len(df_)] = new_row



***
***
# Deep Learning With Keras
### Using SMOTE due to poor results w/o artificial sampling
##### This really feels like overfitting
***
***

In [19]:
model = Sequential()
model.add(Dense(15, input_dim=7, activation='relu'))
#model.add(Conv1D(4, kernel_size=2, activation='relu'))
#model.add(Dropout(0.50))
#model.add(MaxPooling1D(pool_size=2))
#model.add(Flatten())
#model.add(Dense(75, activation='relu'))
#model.add(Dropout(0.25))
#model.add(Dense(15, activation='relu'))
#model.add(Dropout(0.125))
model.add(Dense(7, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:

#######################
#model.predict_classes is depricated since this code was initially written, so I replaced it
#I couldn't send yall broken code~~~
#
#######################


sc = StandardScaler()
X = sc.fit_transform(Bio_X_res)

#these casts were uselesss I think
#X = np.asarray(X).astype('float32')
#Y = np.asarray(Bio_y_res).astype('float32')
                              
model.fit(X, Bio_y_res, epochs=1, batch_size=25)


#predictions = model.predict_classes(Astro_X)
Astro_X = np.asarray(Astro_X).astype('float32')
predictions = (model.predict(Astro_X) > 0.5).astype("float32")

produce_metrics(Astro_y, predictions)
print(confusion_matrix(Astro_y, predictions))
print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")


#predictions = model.predict_classes(Chem_X)
Chem_X = np.asarray(Chem_X).astype('float32')
predictions = (model.predict(Chem_X) > 0.5).astype("float32")

produce_metrics(Chem_y, predictions)
print(confusion_matrix(Chem_y, predictions))
print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")


#predictions = model.predict_classes(Others_X)
Others_X = np.asarray(Others_X).astype('float32')
predictions = (model.predict(Others_X) > 0.5).astype("float32")

produce_metrics(Others_y, predictions)
print(confusion_matrix(Others_y, predictions))

12738/12738 [==============================] - 9s 646us/step - loss: 0.6383 - accuracy: 0.6235
[array([0.93, 0.06]), array([0.93, 0.05]), array([0.93, 0.05]), array([62496.,  4848.])]
[[58328  4168]
 [ 4602   246]]
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
[array([0.94, 0.05]), array([0.97, 0.02]), array([0.96, 0.03]), array([49397.,  2998.])]
[[48000  1397]
 [ 2932    66]]
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
[array([0.96, 0.05]), array([0.99, 0.01]), array([0.97, 0.01]), array([5151.,  233.])]
[[5109   42]
 [ 231    2]]


In [21]:
sc = StandardScaler()
X = sc.fit_transform(Astro_X_res)
#Y = np.asarray(Astro_y_res).astype('float32')

model.fit(X, Astro_y_res, epochs=1, batch_size=25)

#predictions = model.predict_classes(Bio_X)
Bio_X = np.asarray(Bio_X).astype('float32')
predictions = (model.predict(Bio_X) > 0.5).astype("float32")

produce_metrics(Bio_y, predictions)
print(confusion_matrix(Bio_y, predictions))
print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")

#predictions = model.predict_classes(Chem_X)
Chem_X = np.asarray(Chem_X).astype('float32')
predictions = (model.predict(Chem_X) > 0.5).astype("float32")

produce_metrics(Chem_y, predictions)
print(confusion_matrix(Chem_y, predictions))
print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")

#predictions = model.predict_classes(Others_X)
Others_X = np.asarray(Others_X).astype('float32')
predictions = (model.predict(Others_X) > 0.5).astype("float32")

produce_metrics(Others_y, predictions)
print(confusion_matrix(Others_y, predictions))

5000/5000 [==============================] - 3s 646us/step - loss: 0.6859 - accuracy: 0.5436
[array([0.91, 0.08]), array([0.63, 0.34]), array([0.74, 0.13]), array([159216.,  15717.])]
[[99899 59317]
 [10373  5344]]
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
[array([0.95, 0.06]), array([0.35, 0.67]), array([0.52, 0.11]), array([49397.,  2998.])]
[[17486 31911]
 [ 1003  1995]]
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
[array([0.95, 0.04]), array([0.28, 0.67]), array([0.43, 0.08]), array([5151.,  233.])]
[[1438 3713]
 [  78  155]]


In [22]:
sc = StandardScaler()
X = sc.fit_transform(Chem_X_res)
#Y = np.asarray(Chem_y_res).astype('float32')

model.fit(X, Chem_y_res, epochs=1, batch_size=10)

#predictions = model.predict_classes(Bio_X)
Bio_X = np.asarray(Bio_X).astype('float32')
predictions = (model.predict(Bio_X) > 0.5).astype("float32")

produce_metrics(Bio_y, predictions)
print(confusion_matrix(Bio_y, predictions))
print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")

#predictions = model.predict_classes(Astro_X)
Astro_X = np.asarray(Astro_X).astype('float32')
predictions = (model.predict(Astro_X) > 0.5).astype("float32")

produce_metrics(Astro_y, predictions)
print(confusion_matrix(Astro_y, predictions))
print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")

#predictions = model.predict_classes(Others_X)
Others_X = np.asarray(Others_X).astype('float32')
predictions = (model.predict(Others_X) > 0.5).astype("float32")

produce_metrics(Others_y, predictions)
print(confusion_matrix(Others_y, predictions))

9880/9880 [==============================] - 6s 655us/step - loss: 0.6667 - accuracy: 0.5837
[array([0.91, 0.05]), array([1., 0.]), array([0.95, 0.  ]), array([159216.,  15717.])]
[[159004    212]
 [ 15705     12]]
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
[array([0.93, 0.02]), array([1., 0.]), array([0.96, 0.  ]), array([62496.,  4848.])]
[[62447    49]
 [ 4847     1]]
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
[array([0.96, 0.  ]), array([1., 0.]), array([0.98, 0.  ]), array([5151.,  233.])]
[[5146    5]
 [ 233    0]]


***
### Bio -> Astro 
***

In [23]:
df_metrics = pd.DataFrame( columns=['Train_Set', 'Target_Set', 'Model_Type','Neutral_Precision','Insecure_Precision', 'Neutral_Recall', 'Insecure_Recall', 
                                              'Neutral_F', 'Insecure_F', 'Neutral_Support', 'Insecure_Support', 'Neutral_Precision_SMOTE','Insecure_Precision_SMOTE',
                                              'Neutral_Recall_SMOTE', 'Insecure_Recall_SMOTE', 'Neutral_F_SMOTE', 'Insecure_F_SMOTE', 'Neutral_Support_SMOTE', 'Insecure_Support_SMOTE' ])

df_metrics

#puts new entry into table
def add_metrics_row(trainer, tester, clf_type, normals, smotes, df_):
    new_row = [trainer, tester, clf_type]
    for x in normals:
        for y in x:
            new_row.append(y)    
    for x in smotes:
        for y in x:
            new_row.append(y)        
    df_.loc[len(df_)] = new_row


    

In [24]:
clf = DecisionTreeClassifier(random_state=37)

normal_metrics = run_transfer_model(clf, Bio_X, Bio_y, Astro_X, Astro_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Bio_X_res, Bio_y_res, Astro_X, Astro_y)


add_metrics_row("bio", "astro", type(clf), normal_metrics, smote_metrics, df_metrics)

[array([0.93, 0.07]), array([0.9 , 0.09]), array([0.91, 0.08]), array([62496.,  4848.])]
[[56334  6162]
 [ 4394   454]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[array([0.93, 0.07]), array([0.81, 0.18]), array([0.86, 0.1 ]), array([62496.,  4848.])]
[[50509 11987]
 [ 3987   861]]


In [25]:
clf = KNeighborsClassifier(n_neighbors = 547)

normal_metrics = run_transfer_model(clf, Bio_X, Bio_y, Astro_X, Astro_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Bio_X_res, Bio_y_res, Astro_X, Astro_y)

add_metrics_row("bio", "astro", type(clf), normal_metrics, smote_metrics, df_metrics)

[array([0.93, 0.07]), array([0.99, 0.01]), array([0.96, 0.02]), array([62496.,  4848.])]
[[61913   583]
 [ 4802    46]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[array([0.93, 0.07]), array([0.76, 0.25]), array([0.83, 0.11]), array([62496.,  4848.])]
[[47387 15109]
 [ 3637  1211]]


In [26]:
clf = BernoulliNB()

normal_metrics = run_transfer_model(clf, Bio_X, Bio_y, Astro_X, Astro_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Bio_X_res, Bio_y_res, Astro_X, Astro_y)  

add_metrics_row("bio", "astro", type(clf), normal_metrics, smote_metrics, df_metrics)

[array([0.93, 0.  ]), array([1., 0.]), array([0.96, 0.  ]), array([62496.,  4848.])]
[[62496     0]
 [ 4848     0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[array([0.95, 0.08]), array([0.18, 0.87]), array([0.3 , 0.14]), array([62496.,  4848.])]
[[11124 51372]
 [  612  4236]]


/home/adam/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, random_state=1)

normal_metrics = run_transfer_model(clf, Bio_X, Bio_y, Astro_X, Astro_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Bio_X_res, Bio_y_res, Astro_X, Astro_y)

add_metrics_row("bio", "astro", type(clf), normal_metrics, smote_metrics, df_metrics)

/home/adam/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/adam/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[array([0.93, 0.  ]), array([1., 0.]), array([0.96, 0.  ]), array([62496.,  4848.])]
[[62496     0]
 [ 4848     0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[array([0.93, 0.07]), array([0.66, 0.33]), array([0.77, 0.12]), array([62496.,  4848.])]
[[41303 21193]
 [ 3259  1589]]


In [28]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

normal_metrics = run_transfer_model(clf, Bio_X, Bio_y, Astro_X, Astro_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Bio_X_res, Bio_y_res, Astro_X, Astro_y)

add_metrics_row("bio", "astro", type(clf), normal_metrics, smote_metrics, df_metrics)

/home/adam/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[array([0.93, 0.  ]), array([1., 0.]), array([0.96, 0.  ]), array([62496.,  4848.])]
[[62496     0]
 [ 4848     0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[array([0.93, 0.07]), array([0.58, 0.41]), array([0.71, 0.12]), array([62496.,  4848.])]
[[36181 26315]
 [ 2882  1966]]


***
### Bio -> Chem
***

In [29]:
clf = DecisionTreeClassifier(random_state=37)

normal_metrics = run_transfer_model(clf, Bio_X, Bio_y, Chem_X, Chem_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Bio_X_res, Bio_y_res, Chem_X, Chem_y)

add_metrics_row("bio", "chem", type(clf), normal_metrics, smote_metrics, df_metrics)

df_metrics

[array([0.94, 0.06]), array([0.84, 0.16]), array([0.89, 0.09]), array([49397.,  2998.])]
[[41421  7976]
 [ 2506   492]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[array([0.94, 0.07]), array([0.85, 0.18]), array([0.89, 0.1 ]), array([49397.,  2998.])]
[[41905  7492]
 [ 2451   547]]


,Train_Set,Target_Set,Model_Type,Neutral_Precision,Insecure_Precision,Neutral_Recall,Insecure_Recall,Neutral_F,Insecure_F,Neutral_Support,Insecure_Support,Neutral_Precision_SMOTE,Insecure_Precision_SMOTE,Neutral_Recall_SMOTE,Insecure_Recall_SMOTE,Neutral_F_SMOTE,Insecure_F_SMOTE,Neutral_Support_SMOTE,Insecure_Support_SMOTE
0,bio,astro,<class 'sklearn.tree._classes.DecisionTreeClas...,0.93,0.07,0.90,0.09,0.91,0.08,62496.0,4848.0,0.93,0.07,0.81,0.18,0.86,0.10,62496.0,4848.0
1,bio,astro,<class 'sklearn.neighbors._classification.KNei...,0.93,0.07,0.99,0.01,0.96,0.02,62496.0,4848.0,0.93,0.07,0.76,0.25,0.83,0.11,62496.0,4848.0
2,bio,astro,<class 'sklearn.naive_bayes.BernoulliNB'>,0.93,0.00,1.00,0.00,0.96,0.00,62496.0,4848.0,0.95,0.08,0.18,0.87,0.30,0.14,62496.0,4848.0
3,bio,astro,<class 'sklearn.neural_network._multilayer_per...,0.93,0.00,1.00,0.00,0.96,0.00,62496.0,4848.0,0.93,0.07,0.66,0.33,0.77,0.12,62496.0,4848.0
4,bio,astro,<class 'sklearn.ensemble._forest.RandomForestC...,0.93,0.00,1.00,0.00,0.96,0.00,62496.0,4848.0,0.93,0.07,0.58,0.41,0.71,0.12,62496.0,4848.0
5,bio,chem,<class 'sklearn.tree._classes.DecisionTreeClas...,0.94,0.06,0.84,0.16,0.89,0.09,49397.0,2998.0,0.94,0.07,0.85,0.18,0.89,0.10,49397.0,2998.0


In [30]:
clf = KNeighborsClassifier(n_neighbors = 547)

normal_metrics = run_transfer_model(clf, Bio_X, Bio_y, Chem_X, Chem_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Bio_X_res, Bio_y_res, Chem_X, Chem_y)

add_metrics_row("bio", "chem", type(clf), normal_metrics, smote_metrics, df_metrics)

/home/adam/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[array([0.94, 0.  ]), array([1., 0.]), array([0.97, 0.  ]), array([49397.,  2998.])]
[[49397     0]
 [ 2998     0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[array([0.94, 0.06]), array([0.84, 0.17]), array([0.89, 0.09]), array([49397.,  2998.])]
[[41294  8103]
 [ 2486   512]]


In [31]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, random_state=1, max_iter = 1000)

normal_metrics = run_transfer_model(clf, Bio_X, Bio_y, Chem_X, Chem_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Bio_X_res, Bio_y_res, Chem_X, Chem_y)

add_metrics_row("bio", "chem", type(clf), normal_metrics, smote_metrics, df_metrics)

/home/adam/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/adam/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[array([0.94, 0.  ]), array([1., 0.]), array([0.97, 0.  ]), array([49397.,  2998.])]
[[49397     0]
 [ 2998     0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[array([0.94, 0.04]), array([0.69, 0.23]), array([0.79, 0.07]), array([49397.,  2998.])]
[[34081 15316]
 [ 2314   684]]


In [32]:
clf = BernoulliNB()

normal_metrics = run_transfer_model(clf, Bio_X, Bio_y, Chem_X, Chem_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Bio_X_res, Bio_y_res, Chem_X, Chem_y)

add_metrics_row("bio", "chem", type(clf), normal_metrics, smote_metrics, df_metrics)

[array([0.94, 0.  ]), array([1., 0.]), array([0.97, 0.  ]), array([49397.,  2998.])]
[[49397     0]
 [ 2998     0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[array([0.95, 0.06]), array([0.15, 0.86]), array([0.26, 0.11]), array([49397.,  2998.])]
[[ 7473 41924]
 [  407  2591]]


/home/adam/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

normal_metrics = run_transfer_model(clf, Bio_X, Bio_y, Chem_X, Chem_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Bio_X_res, Bio_y_res, Chem_X, Chem_y)

add_metrics_row("bio", "chem", type(clf), normal_metrics, smote_metrics, df_metrics)

/home/adam/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[array([0.94, 0.  ]), array([1., 0.]), array([0.97, 0.  ]), array([49397.,  2998.])]
[[49397     0]
 [ 2998     0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[array([0.94, 0.06]), array([0.8 , 0.23]), array([0.87, 0.1 ]), array([49397.,  2998.])]
[[39453  9944]
 [ 2323   675]]


***
### Bio -> Others
***

In [34]:
clf = DecisionTreeClassifier(random_state=37)

normal_metrics = run_transfer_model(clf, Bio_X, Bio_y, Others_X, Others_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Bio_X_res, Bio_y_res, Others_X, Others_y)

add_metrics_row("bio", "others", type(clf), normal_metrics, smote_metrics, df_metrics)

[array([0.96, 0.03]), array([0.86, 0.11]), array([0.9 , 0.05]), array([5151.,  233.])]
[[4423  728]
 [ 208   25]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[array([0.96, 0.05]), array([0.87, 0.16]), array([0.91, 0.08]), array([5151.,  233.])]
[[4470  681]
 [ 196   37]]


In [35]:
clf = KNeighborsClassifier(n_neighbors = 547)

normal_metrics = run_transfer_model(clf, Bio_X, Bio_y, Others_X, Others_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Bio_X_res, Bio_y_res, Others_X, Others_y)

add_metrics_row("bio", "others", type(clf), normal_metrics, smote_metrics, df_metrics)

/home/adam/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[array([0.96, 0.  ]), array([1., 0.]), array([0.98, 0.  ]), array([5151.,  233.])]
[[5151    0]
 [ 233    0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[array([0.96, 0.05]), array([0.86, 0.17]), array([0.91, 0.08]), array([5151.,  233.])]
[[4454  697]
 [ 194   39]]


In [36]:
clf = BernoulliNB()

normal_metrics = run_transfer_model(clf, Bio_X, Bio_y, Others_X, Others_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Bio_X_res, Bio_y_res, Others_X, Others_y)

add_metrics_row("bio", "others", type(clf), normal_metrics, smote_metrics, df_metrics)

[array([0.96, 0.  ]), array([1., 0.]), array([0.98, 0.  ]), array([5151.,  233.])]
[[5151    0]
 [ 233    0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[array([0.98, 0.05]), array([0.16, 0.92]), array([0.28, 0.09]), array([5151.,  233.])]
[[ 837 4314]
 [  19  214]]


/home/adam/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [37]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, random_state=1)

normal_metrics = run_transfer_model(clf, Bio_X, Bio_y, Others_X, Others_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Bio_X_res, Bio_y_res, Others_X, Others_y)

add_metrics_row("bio", "others", type(clf), normal_metrics, smote_metrics, df_metrics)

/home/adam/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/adam/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[array([0.96, 0.  ]), array([1., 0.]), array([0.98, 0.  ]), array([5151.,  233.])]
[[5151    0]
 [ 233    0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[array([0.96, 0.05]), array([0.94, 0.06]), array([0.95, 0.05]), array([5151.,  233.])]
[[4847  304]
 [ 218   15]]


In [38]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

normal_metrics = run_transfer_model(clf, Bio_X, Bio_y, Others_X, Others_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Bio_X_res, Bio_y_res, Others_X, Others_y)

add_metrics_row("bio", "others", type(clf), normal_metrics, smote_metrics, df_metrics)

/home/adam/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[array([0.96, 0.  ]), array([1., 0.]), array([0.98, 0.  ]), array([5151.,  233.])]
[[5151    0]
 [ 233    0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[array([0.96, 0.07]), array([0.83, 0.26]), array([0.89, 0.11]), array([5151.,  233.])]
[[4287  864]
 [ 172   61]]


***
### Astro -> Bio
***

In [39]:
clf = DecisionTreeClassifier(random_state=37)

normal_metrics = run_transfer_model(clf, Astro_X, Astro_y, Bio_X, Bio_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Astro_X_res, Astro_y_res, Bio_X, Bio_y)

add_metrics_row("astro", "bio", type(clf), normal_metrics, smote_metrics, df_metrics)

[array([0.91, 0.07]), array([0.87, 0.1 ]), array([0.89, 0.09]), array([159216.,  15717.])]
[[138739  20477]
 [ 14068   1649]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[array([0.91, 0.08]), array([0.87, 0.12]), array([0.89, 0.1 ]), array([159216.,  15717.])]
[[138703  20513]
 [ 13835   1882]]


In [40]:
clf = KNeighborsClassifier(n_neighbors = 547)

normal_metrics = run_transfer_model(clf, Astro_X, Astro_y, Bio_X, Bio_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Astro_X_res, Astro_y_res, Bio_X, Bio_y)

add_metrics_row("astro", "bio", type(clf), normal_metrics, smote_metrics, df_metrics)

/home/adam/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[array([0.91, 0.  ]), array([1., 0.]), array([0.95, 0.  ]), array([159216.,  15717.])]
[[159216      0]
 [ 15717      0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[array([0.9 , 0.08]), array([0.54, 0.39]), array([0.68, 0.13]), array([159216.,  15717.])]
[[86134 73082]
 [ 9534  6183]]


In [41]:
clf = BernoulliNB()

normal_metrics = run_transfer_model(clf, Astro_X, Astro_y, Bio_X, Bio_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Astro_X_res, Astro_y_res, Bio_X, Bio_y)

add_metrics_row("astro", "bio", type(clf), normal_metrics, smote_metrics, df_metrics)

[array([0.91, 0.  ]), array([1., 0.]), array([0.95, 0.  ]), array([159216.,  15717.])]


/home/adam/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[[159216      0]
 [ 15717      0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[array([0.93, 0.09]), array([0.17, 0.86]), array([0.29, 0.17]), array([159216.,  15717.])]
[[ 27246 131970]
 [  2197  13520]]


In [42]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, random_state=1)

normal_metrics = run_transfer_model(clf, Astro_X, Astro_y, Bio_X, Bio_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Astro_X_res, Astro_y_res, Bio_X, Bio_y)

add_metrics_row("astro", "bio", type(clf), normal_metrics, smote_metrics, df_metrics)

/home/adam/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[array([0.91, 0.06]), array([0.98, 0.01]), array([0.94, 0.02]), array([159216.,  15717.])]
[[156227   2989]
 [ 15531    186]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[array([0.91, 0.09]), array([0.18, 0.83]), array([0.31, 0.16]), array([159216.,  15717.])]
[[ 29182 130034]
 [  2731  12986]]


In [43]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

normal_metrics = run_transfer_model(clf, Astro_X, Astro_y, Bio_X, Bio_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Astro_X_res, Astro_y_res, Bio_X, Bio_y)

add_metrics_row("astro", "bio", type(clf), normal_metrics, smote_metrics, df_metrics)

/home/adam/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[array([0.91, 0.  ]), array([1., 0.]), array([0.95, 0.  ]), array([159216.,  15717.])]
[[159216      0]
 [ 15717      0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[array([0.9 , 0.08]), array([0.37, 0.59]), array([0.52, 0.15]), array([159216.,  15717.])]
[[ 58712 100504]
 [  6455   9262]]


***
### Astro -> Chem
***

In [44]:
clf = DecisionTreeClassifier(random_state=37)

normal_metrics = run_transfer_model(clf, Astro_X, Astro_y, Chem_X, Chem_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Astro_X_res, Astro_y_res, Chem_X, Chem_y)

add_metrics_row("astro", "chem", type(clf), normal_metrics, smote_metrics, df_metrics)

[array([0.94, 0.06]), array([0.8, 0.2]), array([0.86, 0.09]), array([49397.,  2998.])]
[[39294 10103]
 [ 2398   600]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[array([0.94, 0.06]), array([0.84, 0.17]), array([0.89, 0.09]), array([49397.,  2998.])]
[[41278  8119]
 [ 2483   515]]


In [45]:
clf = KNeighborsClassifier(n_neighbors = 547)

normal_metrics = run_transfer_model(clf, Astro_X, Astro_y, Chem_X, Chem_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Astro_X_res, Astro_y_res, Chem_X, Chem_y)

add_metrics_row("astro", "chem", type(clf), normal_metrics, smote_metrics, df_metrics)

/home/adam/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[array([0.94, 0.  ]), array([1., 0.]), array([0.97, 0.  ]), array([49397.,  2998.])]
[[49397     0]
 [ 2998     0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[array([0.95, 0.06]), array([0.56, 0.48]), array([0.71, 0.11]), array([49397.,  2998.])]
[[27897 21500]
 [ 1550  1448]]


In [46]:
clf = BernoulliNB()

normal_metrics = run_transfer_model(clf, Astro_X, Astro_y, Chem_X, Chem_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Astro_X_res, Astro_y_res, Chem_X, Chem_y)

add_metrics_row("astro", "chem", type(clf), normal_metrics, smote_metrics, df_metrics)

[array([0.94, 0.  ]), array([1., 0.]), array([0.97, 0.  ]), array([49397.,  2998.])]
[[49397     0]
 [ 2998     0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[array([0.95, 0.06]), array([0.15, 0.86]), array([0.26, 0.11]), array([49397.,  2998.])]
[[ 7473 41924]
 [  407  2591]]


/home/adam/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [47]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, random_state=1)

normal_metrics = run_transfer_model(clf, Astro_X, Astro_y, Chem_X, Chem_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Astro_X_res, Astro_y_res, Chem_X, Chem_y)

add_metrics_row("astro", "chem", type(clf), normal_metrics, smote_metrics, df_metrics)

/home/adam/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[array([0.94, 0.06]), array([0.95, 0.06]), array([0.94, 0.06]), array([49397.,  2998.])]
[[46711  2686]
 [ 2829   169]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[array([0.94, 0.05]), array([0.35, 0.62]), array([0.51, 0.1 ]), array([49397.,  2998.])]
[[17469 31928]
 [ 1144  1854]]


In [48]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

normal_metrics = run_transfer_model(clf, Astro_X, Astro_y, Chem_X, Chem_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Astro_X_res, Astro_y_res, Chem_X, Chem_y)

add_metrics_row("astro", "chem", type(clf), normal_metrics, smote_metrics, df_metrics)

/home/adam/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[array([0.94, 0.  ]), array([1., 0.]), array([0.97, 0.  ]), array([49397.,  2998.])]
[[49397     0]
 [ 2998     0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[array([0.95, 0.06]), array([0.23, 0.81]), array([0.37, 0.11]), array([49397.,  2998.])]
[[11429 37968]
 [  563  2435]]


***
### Astro -> Others
***

In [49]:
clf = DecisionTreeClassifier(random_state=37)

normal_metrics = run_transfer_model(clf, Astro_X, Astro_y, Others_X, Others_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Astro_X_res, Astro_y_res, Others_X, Others_y)

add_metrics_row("astro", "others", type(clf), normal_metrics, smote_metrics, df_metrics)

[array([0.95, 0.04]), array([0.79, 0.18]), array([0.86, 0.06]), array([5151.,  233.])]
[[4064 1087]
 [ 192   41]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[array([0.96, 0.05]), array([0.86, 0.15]), array([0.9 , 0.07]), array([5151.,  233.])]
[[4408  743]
 [ 197   36]]


In [50]:
clf = KNeighborsClassifier(n_neighbors = 547)

normal_metrics = run_transfer_model(clf, Astro_X, Astro_y, Others_X, Others_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Astro_X_res, Astro_y_res, Others_X, Others_y)

add_metrics_row("astro", "others", type(clf), normal_metrics, smote_metrics, df_metrics)

/home/adam/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[array([0.96, 0.  ]), array([1., 0.]), array([0.98, 0.  ]), array([5151.,  233.])]
[[5151    0]
 [ 233    0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[array([0.95, 0.04]), array([0.33, 0.62]), array([0.49, 0.08]), array([5151.,  233.])]
[[1710 3441]
 [  89  144]]


In [51]:
clf = BernoulliNB()

normal_metrics = run_transfer_model(clf, Astro_X, Astro_y, Others_X, Others_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Astro_X_res, Astro_y_res, Others_X, Others_y)

add_metrics_row("astro", "others", type(clf), normal_metrics, smote_metrics, df_metrics)

[array([0.96, 0.  ]), array([1., 0.]), array([0.98, 0.  ]), array([5151.,  233.])]
[[5151    0]
 [ 233    0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[array([0.98, 0.05]), array([0.16, 0.92]), array([0.28, 0.09]), array([5151.,  233.])]
[[ 837 4314]
 [  19  214]]


/home/adam/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [52]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, random_state=1, max_iter = 1000)

normal_metrics = run_transfer_model(clf, Astro_X, Astro_y, Others_X, Others_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Astro_X_res, Astro_y_res, Others_X, Others_y)

add_metrics_row("astro", "others", type(clf), normal_metrics, smote_metrics, df_metrics)

/home/adam/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[array([0.96, 0.03]), array([0.99, 0.  ]), array([0.97, 0.01]), array([5151.,  233.])]
[[5119   32]
 [ 232    1]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[array([0.96, 0.06]), array([0.65, 0.47]), array([0.78, 0.1 ]), array([5151.,  233.])]
[[3370 1781]
 [ 124  109]]


In [53]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

normal_metrics = run_transfer_model(clf, Astro_X, Astro_y, Others_X, Others_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Astro_X_res, Astro_y_res, Others_X, Others_y)

add_metrics_row("astro", "others", type(clf), normal_metrics, smote_metrics, df_metrics)

/home/adam/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[array([0.96, 0.  ]), array([1., 0.]), array([0.98, 0.  ]), array([5151.,  233.])]
[[5151    0]
 [ 233    0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[array([0.98, 0.05]), array([0.24, 0.89]), array([0.38, 0.1 ]), array([5151.,  233.])]
[[1232 3919]
 [  25  208]]


***
### Chem -> Bio
***

In [54]:
clf = DecisionTreeClassifier(random_state=37)

normal_metrics = run_transfer_model(clf, Chem_X, Chem_y, Bio_X, Bio_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Chem_X_res, Chem_y_res, Bio_X, Bio_y)

add_metrics_row("chem", "bio", type(clf), normal_metrics, smote_metrics, df_metrics)

[array([0.91, 0.08]), array([0.92, 0.07]), array([0.91, 0.07]), array([159216.,  15717.])]
[[146069  13147]
 [ 14648   1069]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[array([0.91, 0.08]), array([0.9 , 0.09]), array([0.91, 0.08]), array([159216.,  15717.])]
[[143872  15344]
 [ 14356   1361]]


In [55]:
clf = KNeighborsClassifier(n_neighbors = 547)

normal_metrics = run_transfer_model(clf, Chem_X, Chem_y, Bio_X, Bio_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Chem_X_res, Chem_y_res, Bio_X, Bio_y)

add_metrics_row("chem", "bio", type(clf), normal_metrics, smote_metrics, df_metrics)

/home/adam/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[array([0.91, 0.  ]), array([1., 0.]), array([0.95, 0.  ]), array([159216.,  15717.])]
[[159216      0]
 [ 15717      0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[array([0.91, 0.09]), array([0.63, 0.36]), array([0.74, 0.14]), array([159216.,  15717.])]
[[99843 59373]
 [10075  5642]]


In [56]:
clf = BernoulliNB()

normal_metrics = run_transfer_model(clf, Chem_X, Chem_y, Bio_X, Bio_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Chem_X_res, Chem_y_res, Bio_X, Bio_y)

add_metrics_row("chem", "bio", type(clf), normal_metrics, smote_metrics, df_metrics)

[array([0.91, 0.  ]), array([1., 0.]), array([0.95, 0.  ]), array([159216.,  15717.])]


/home/adam/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[[159216      0]
 [ 15717      0]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


KeyboardInterrupt: 

In [ ]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, random_state=1)

normal_metrics = run_transfer_model(clf, Chem_X, Chem_y, Bio_X, Bio_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Chem_X_res, Chem_y_res, Bio_X, Bio_y)

add_metrics_row("chem", "bio", type(clf), normal_metrics, smote_metrics, df_metrics)

In [ ]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

normal_metrics = run_transfer_model(clf, Chem_X, Chem_y, Bio_X, Bio_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Chem_X_res, Chem_y_res, Bio_X, Bio_y)

add_metrics_row("chem", "bio", type(clf), normal_metrics, smote_metrics, df_metrics)

***
### Chem -> Astro
***

In [ ]:
clf = DecisionTreeClassifier(random_state=37)

normal_metrics = run_transfer_model(clf, Chem_X, Chem_y, Astro_X, Astro_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Chem_X_res, Chem_y_res, Astro_X, Astro_y)

add_metrics_row("chem", "astro", type(clf), normal_metrics, smote_metrics, df_metrics)

In [ ]:
clf = KNeighborsClassifier(n_neighbors = 547)

normal_metrics = run_transfer_model(clf, Chem_X, Chem_y, Astro_X, Astro_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Chem_X_res, Chem_y_res, Astro_X, Astro_y)

add_metrics_row("chem", "astro", type(clf), normal_metrics, smote_metrics, df_metrics)

In [ ]:
clf = BernoulliNB()

normal_metrics = run_transfer_model(clf, Chem_X, Chem_y, Astro_X, Astro_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Chem_X_res, Chem_y_res, Astro_X, Astro_y)

add_metrics_row("chem", "astro", type(clf), normal_metrics, smote_metrics, df_metrics)

In [ ]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, random_state=1)

normal_metrics = run_transfer_model(clf, Chem_X, Chem_y, Astro_X, Astro_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Chem_X_res, Chem_y_res, Astro_X, Astro_y)

add_metrics_row("chem", "astro", type(clf), normal_metrics, smote_metrics, df_metrics)

In [ ]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

normal_metrics = run_transfer_model(clf, Chem_X, Chem_y, Astro_X, Astro_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Chem_X_res, Chem_y_res, Astro_X, Astro_y)

add_metrics_row("chem", "astro", type(clf), normal_metrics, smote_metrics, df_metrics)

***
### Chem -> Others
***

In [ ]:
clf = DecisionTreeClassifier(random_state=37)

normal_metrics = run_transfer_model(clf, Chem_X, Chem_y, Others_X, Others_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Chem_X_res, Chem_y_res, Others_X, Others_y)

add_metrics_row("chem", "others", type(clf), normal_metrics, smote_metrics, df_metrics)

In [ ]:
clf = KNeighborsClassifier(n_neighbors = 547)

normal_metrics = run_transfer_model(clf, Chem_X, Chem_y, Others_X, Others_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Chem_X_res, Chem_y_res, Others_X, Others_y)

add_metrics_row("chem", "others", type(clf), normal_metrics, smote_metrics, df_metrics)

In [ ]:
clf = BernoulliNB()

normal_metrics = run_transfer_model(clf, Chem_X, Chem_y, Others_X, Others_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Chem_X_res, Chem_y_res, Others_X, Others_y)

add_metrics_row("chem", "others", type(clf), normal_metrics, smote_metrics, df_metrics)

In [ ]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, random_state=1)

normal_metrics = run_transfer_model(clf, Chem_X, Chem_y, Others_X, Others_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Chem_X_res, Chem_y_res, Others_X, Others_y)

add_metrics_row("chem", "others", type(clf), normal_metrics, smote_metrics, df_metrics)

In [ ]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

normal_metrics = run_transfer_model(clf, Chem_X, Chem_y, Others_X, Others_y)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
smote_metrics = run_transfer_model(clf, Chem_X_res, Chem_y_res, Others_X, Others_y)

add_metrics_row("chem", "others", type(clf), normal_metrics, smote_metrics, df_metrics)

In [ ]:
df_metrics

In [ ]:
df_metrics.to_csv('metrics.csv')